In [1]:
from math import floor

import music21 as m21
import os

In [3]:
for i in range(0, 2048 - 256, 256 //2):
    print(f'input from {i} to {i+256}')
    print(f'target from {i+1} to {i+1+256}')
    print()

input from 0 to 256
target from 1 to 257

input from 128 to 384
target from 129 to 385

input from 256 to 512
target from 257 to 513

input from 384 to 640
target from 385 to 641

input from 512 to 768
target from 513 to 769

input from 640 to 896
target from 641 to 897

input from 768 to 1024
target from 769 to 1025

input from 896 to 1152
target from 897 to 1153

input from 1024 to 1280
target from 1025 to 1281

input from 1152 to 1408
target from 1153 to 1409

input from 1280 to 1536
target from 1281 to 1537

input from 1408 to 1664
target from 1409 to 1665

input from 1536 to 1792
target from 1537 to 1793

input from 1664 to 1920
target from 1665 to 1921



In [2]:
# load the list of any artists files
BASE_MUSIC_PATH = '../data/music/'
ISOLATE_MIDI_TRACKS = '../midi/'
ARTIST_NAME = 'Guns_N_Roses'
for file in os.listdir(BASE_MUSIC_PATH + ARTIST_NAME):
    print(file)

Knockin_on_Heaven_Door.3.mid
Knockin_on_Heaven_Door.2.mid
Get_in_the_Ring.mid
Sweet_Child_O_Mine.1.mid
Knockin_on_Heaven_Door.1.mid
Knockin_on_Heaven_Door.4.mid
Knockin_on_Heaven_Door.mid
You_Could_Be_Mine.mid
November_Rain.mid
Civil_War.1.mid
Rocket_Queen.mid
Mr._Brownstone.mid
Yesterday_s.mid
Don_t_Cry.mid
Dust_N_Bones.mid
Locomotive.mid
Paradise_City.1.mid
Civil_War.mid
Paradise_City.mid
Welcome_to_the_Jungle.2.mid
You_Could_Be_Mine.1.mid
Welcome_to_the_Jungle.1.mid
Welcome_to_the_Jungle.mid
Coma.mid
November_Rain.4.mid
Since_I_Don_t_Have_You.mid
November_Rain.1.mid
Estranged.mid
Nightrain.mid
Live_Let_Die.mid
Sweet_Child_O_Mine.mid
Don_t_Cry.1.mid
Back_Off_Bitch.mid
Dust_N_Bones.1.mid
November_Rain.3.mid
November_Rain.2.mid
Patience.mid
Live_Let_Die.1.mid
Patience.1.mid
Patience.2.mid


In [3]:
# midis = []
# for file in os.listdir(BASE_MUSIC_PATH + ARTIST_NAME):
#     song_name = file.split('.mid')[0].replace('_',' ')
#     print(f"Song: {song_name}")
#     midi = m21.converter.parse(f'{BASE_MUSIC_PATH}{ARTIST_NAME}/{file}')
#     midi.songName = song_name # just adding this as a property on the Music21Object, yes, this is possible
#     midis.append(midi)
midi = m21.converter.parse(f'{BASE_MUSIC_PATH}{ARTIST_NAME}/Don_t_Cry.mid')

In [6]:
# # Testing out MIDI files
# for midi in midis:
#     print(midi.songName, midi.duration)

In [7]:
# for score in midis[:1]:
#     songs = m21.instrument.partitionByInstrument(score)
#     parts = songs.parts
#
#     eg_count = 0
#     for part in parts:
#         print(part, part.partAbbreviation)
#         eg_count += 1 if "Elec Gtr" in part.partAbbreviation else 0
#
#     print(score.songName, len(parts), f"electric guitar track count {eg_count}")



music21.instrument.partitionByInstrument() groups by instrument types, not raw MIDI tracks. So if multiple tracks are labeled with the same instrument (e.g., “Guitar”), music21 may merge them into one part.

If you want to split the MIDI file strictly by raw track/channel (as seen in your DAW) — not merged by instrument — you’ll want to access the individual Part or Track objects directly, without partitionByInstrument.

In [5]:
for i, part in enumerate(midi.parts):
    print(f"Track {i}: {part.partName}, Instrument: {part.getInstrument().instrumentName}")

Track 0: Backing vocals | Don't, Instrument: None
Track 1: Lead vocal | Cry, Instrument: None
Track 2: Gtr. I | Guns n' Roses, Instrument: None
Track 3: Gtr. II, Instrument: None
Track 4: Gtr. III, Instrument: None
Track 5: Gtr. IV, Instrument: None
Track 6: Gtr. V, Instrument: None
Track 7: Lead double, Instrument: None
Track 8: Bass, Instrument: None


In [9]:
songs = m21.instrument.partitionByInstrument(midi)
songs

<music21.stream.Score 0x117227800>

In [10]:
for part in songs.parts:
    print(part)

<music21.stream.Part Choir>
<music21.stream.Part Voice>
<music21.stream.Part Electric Guitar>
<music21.stream.Part Gtr. V>
<music21.stream.Part Baritone Saxophone>
<music21.stream.Part Bass>
<music21.stream.Part Acoustic Bass>


In [11]:
electric_guitar = songs.parts[3]
notes = electric_guitar.notesAndRests.stream()
for pick in notes.recurse():
    # print(pick, pick.__class__)
    if isinstance(pick, (m21.note.Note, m21.note.Rest, m21.chord.Chord)):
        bar_number = pick.getContextByClass(m21.stream.Measure).measureNumber
        if bar_number > 10:
            break
    if pick.__class__ == m21.note.Note:
        pitch = pick.pitch
        duration = pick.duration
        volume = pick.volume
        print(f"Bar {bar_number}; Note {pitch.nameWithOctave}; Duration {duration.type}; Velocity: {volume.velocity}")
    elif pick.__class__ == m21.chord.Chord:
        duration = pick.duration
        volume = pick.volume
        chord_notes = ' '.join([n.pitch.nameWithOctave for n in pick.notes])
        print(f"Bar {bar_number}; Chord {chord_notes}; Duration {duration.type}; Velocity: {volume.velocity}")
    elif pick.__class__ == m21.note.Rest:
        duration = pick.duration
        print(f"Bar {bar_number}; Rest; Duration {duration.type}")


In [13]:
import numpy as np
import math
samples = np.arange(0,127, math.ceil(127/16))
len(samples)

16

In [14]:
from music21 import instrument
electric_guitar_part = None
for part in midi.parts:
    # Attempt to get any defined instrument objects
    instrs = part.getElementsByClass(instrument.Instrument)
    if instrs:
        # Check the name of the first instrument in this part
        # (some MIDI files have multiple instruments in one part, but that is less common)
        first_instr = instrs[0]
        # For General MIDI, "Electric Guitar" might appear in instrumentName or some variation
        # e.g. "Electric Guitar (jazz)", "Overdriven Guitar", etc.
        if "Electric Guitar" in (first_instr.instrumentName or ""):
            electric_guitar_part = part
            print("part", part)
            break

part <music21.stream.Part Electric Guitar>


In [7]:
electric_guitar_part = midi.parts[6]
electric_guitar_part

<music21.stream.Part 0x11b79b770>

In [8]:
# Save it as a new MIDI file temporarily
electric_guitar_part.write('midi', fp=f'{ISOLATE_MIDI_TRACKS}Don_t_Cry_Electric_Guitar.mid')

'../midi/Don_t_Cry_Electric_Guitar.mid'

In [9]:
from miditok import REMI, TokenizerConfig
from symusic import Score

config = TokenizerConfig(num_velocities=16, use_chords=True, use_programs=True, use_rests=True)
tokenizer = REMI(config)

# full song MIDI
# midit = Score(f'{BASE_MUSIC_PATH}{ARTIST_NAME}/Don_t_Cry.mid')

# electric guitar part
midit = Score(f'{ISOLATE_MIDI_TRACKS}Don_t_Cry_Electric_Guitar.mid')

tokseq = tokenizer(midit)
tokens, ids = tokseq.tokens, tokseq.ids
converted_back_midi = tokenizer(tokseq)

tokens[:4], ids[:4]

/opt/miniconda3/envs/music-llm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/miniconda3/envs/music-llm/lib/python3.12/site-packages/miditok/tokenizations/remi.py:88: UserWarning: Attribute controls are not compatible with 'config.one_token_stream_for_programs' and multi-vocabulary tokenizers. Disabling them from the config.
  super().__init__(tokenizer_config, params)


(['Rest_12.0.2', 'Rest_12.0.2', 'Rest_12.0.2', 'Rest_12.0.2'],
 [311, 311, 311, 311])

In [10]:
from music21 import tempo, meter

# Get tempo (returns a MetronomeMark object)
tempo_markings = electric_guitar_part.recurse().getElementsByClass(tempo.MetronomeMark)
for t in tempo_markings:
    print("Tempo:", t.number, "BPM")

# Get time signature (returns TimeSignature objects)
time_signatures = electric_guitar_part.recurse().getElementsByClass(meter.TimeSignature)
for ts in time_signatures:
    print("Time Signature:", ts.ratioString)

Tempo: 62 BPM
Time Signature: 4/4
Time Signature: 2/4
Time Signature: 4/4
Time Signature: 2/4
Time Signature: 4/4


In [11]:
# vocabulary of tokenizer
itom = {i:n for i, n in enumerate(tokenizer.vocab)}

In [12]:
# function to print these tokens
def print_tokens(tokens):
    for i, token in enumerate(tokens):
        if isinstance(token, int):
            print(f"Token {i}: {itom[token]}")
        else:
            print(f"Token {i}: {token}")

In [24]:
tokens[:50]

['Rest_12.0.2',
 'Rest_12.0.2',
 'Rest_12.0.2',
 'Rest_12.0.2',
 'Rest_12.0.2',
 'Rest_12.0.2',
 'Rest_12.0.2',
 'Rest_12.0.2',
 'Rest_12.0.2',
 'Rest_12.0.2',
 'Rest_12.0.2',
 'Rest_12.0.2',
 'Rest_4.0.2',
 'Position_0',
 'Program_30',
 'Pitch_81',
 'Velocity_63',
 'Duration_3.0.8',
 'Program_30',
 'Pitch_69',
 'Velocity_63',
 'Duration_3.0.8',
 'Position_24',
 'Program_30',
 'Pitch_76',
 'Velocity_63',
 'Duration_1.0.8',
 'Bar_None',
 'Position_0',
 'Program_30',
 'Pitch_74',
 'Velocity_63',
 'Duration_1.2.8',
 'Position_10',
 'Program_30',
 'Pitch_74',
 'Velocity_63',
 'Duration_0.6.8',
 'Position_16',
 'Program_30',
 'Pitch_74',
 'Velocity_63',
 'Duration_0.4.8',
 'Position_20',
 'Program_30',
 'Pitch_79',
 'Velocity_63',
 'Duration_0.6.8',
 'Position_26',
 'Program_30']

In [33]:
# for now, this tries to remove any repeated Rest tokens
# the solo need not include the rests as they obviously start much later in most cases
solo_tokens = []
has_position_showed = False
for i in range(1, len(tokens)):
    current_token = tokens[i]
    prev_token = tokens[i - 1]
    current_token_type = current_token.split('_')[0]
    prev_token_type = prev_token.split('_')[0]
    if current_token_type == 'Position' or prev_token_type == 'Position':
        # lets stop if we see a position token, mostly means the solo has started
        has_position_showed =  True
        continue
    elif not has_position_showed and prev_token_type == current_token_type and current_token_type == 'Rest':
        pass
    else:
        solo_tokens.append(current_token)

solo_tokens[:100]

['Pitch_81',
 'Velocity_63',
 'Duration_3.0.8',
 'Program_30',
 'Pitch_69',
 'Velocity_63',
 'Duration_3.0.8',
 'Pitch_76',
 'Velocity_63',
 'Duration_1.0.8',
 'Bar_None',
 'Pitch_74',
 'Velocity_63',
 'Duration_1.2.8',
 'Pitch_74',
 'Velocity_63',
 'Duration_0.6.8',
 'Pitch_74',
 'Velocity_63',
 'Duration_0.4.8',
 'Pitch_79',
 'Velocity_63',
 'Duration_0.6.8',
 'Pitch_74',
 'Velocity_63',
 'Duration_0.6.8',
 'Bar_None',
 'Pitch_74',
 'Velocity_63',
 'Duration_1.0.8',
 'Pitch_77',
 'Velocity_63',
 'Duration_0.2.8',
 'Pitch_77',
 'Velocity_63',
 'Duration_0.2.8',
 'Pitch_74',
 'Velocity_63',
 'Duration_0.4.8',
 'Pitch_72',
 'Velocity_63',
 'Duration_0.4.8',
 'Pitch_69',
 'Velocity_63',
 'Duration_0.4.8',
 'Pitch_67',
 'Velocity_63',
 'Duration_0.4.8',
 'Pitch_62',
 'Velocity_63',
 'Duration_0.4.8',
 'Pitch_60',
 'Velocity_63',
 'Duration_0.4.8',
 'Bar_None',
 'Pitch_62',
 'Velocity_63',
 'Duration_0.4.8',
 'Program_30',
 'Pitch_74',
 'Velocity_63',
 'Duration_0.4.8',
 'Pitch_64',
 'Velo

In [31]:
m21.pitch.Pitch(midi=74).nameWithOctave

'D5'

In [18]:
config.beat_res

{(0, 4): 8, (4, 12): 4}